# WeatherPy

---

In [4]:
# load dependencies
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

# citipy will help determine cities based on latitude and longitude data
from citipy import citipy

# import OpenWeatherMap API key
from api_keys import weather_api_key

### Generate the Cities List by Using the `citipy` Library

In [6]:
# instantiate an empty list for holding lat/lng combinations, and one for holding nearest city names
lat_lngs = []
cities = []

# set a range of latitudes and longitudes for random generation
lat_range = (-90, 90)
lng_range = (-180, 180)

# create a set of 1500 random lat/lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)
lat_lngs

In [7]:
# identify the nearest city for each lat/lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # only add new cities to our list
    if city not in cities:
        cities.append(city)

# print the city count
print(f"Number of cities in the list: {len(cities)}")

Number of cities in the list: 586


### Showcase the Relationship Between Weather Attributes and Latitude

In [25]:
# retrieve weather attributes for relevant cities
api_url = 'https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lng}&appid={API_key}'
city_weather_data = []

for i, city in enumerate(cities):
    response = requests.get(api_url.format(lat=lats[i], lng=lngs[i], API_key=weather_api_key))
    data = response.json()
    
    lat_lng_dict = data['coord']
    print('Lat/Long: ', lat_lng_dict)
    weather_dict = data['weather']
    print('Weather: ', weather_dict)
    base_dict = data['base']
    print('Base: ', base_dict)
    main_dict = data['main']
    print('Main: ', main_dict)
    vis_dict = data['visibility']
    print('Vis: ', vis_dict)
    wind_dict = data['wind']
    print('Wind: ', wind_dict)
    clouds_dict = data['clouds']
    print('Clouds: ', clouds_dict)
    dt_dict = data['dt']
    print('DT: ', dt_dict)
    sys_dict = data['sys']
    print('Sys: ', sys_dict)
    timezone_dict = data['timezone']
    print('Timezone: ', timezone_dict)
    id_dict = data['id']
    print('ID: ', id_dict)
    name_dict = data['name']
    print('Name: ', name_dict)
    cod_dict = data['cod']
    print('Cod: ', cod_dict)

Lat/Long:  {'lon': -158.8196, 'lat': -87.2901}
Weather:  [{'id': 801, 'main': 'Clouds', 'description': 'few clouds', 'icon': '02n'}]
Base:  stations
Main:  {'temp': 228.03, 'feels_like': 221.03, 'temp_min': 228.03, 'temp_max': 228.03, 'pressure': 1020, 'humidity': 90, 'sea_level': 1020, 'grnd_level': 686}
Vis:  10000
Wind:  {'speed': 10.33, 'deg': 140, 'gust': 16.79}
Clouds:  {'all': 14}
DT:  1681283515
Sys:  {'sunrise': 0, 'sunset': 0}
Timezone:  43200
ID:  0
Name:  
Cod:  200
Lat/Long:  {'lon': -54.4889, 'lat': 40.0284}
Weather:  [{'id': 800, 'main': 'Clear', 'description': 'clear sky', 'icon': '01n'}]
Base:  stations
Main:  {'temp': 286.8, 'feels_like': 285.79, 'temp_min': 286.8, 'temp_max': 286.8, 'pressure': 1010, 'humidity': 60, 'sea_level': 1010, 'grnd_level': 1010}
Vis:  10000
Wind:  {'speed': 12.86, 'deg': 276, 'gust': 14.25}
Clouds:  {'all': 5}
DT:  1681283515
Sys:  {'sunrise': 1681290292, 'sunset': 1681337537}
Timezone:  -14400
ID:  0
Name:  
Cod:  200
Lat/Long:  {'lon': -15

KeyboardInterrupt: 

#### Latitude vs. Temperature

#### Latitude vs. Humidity

#### Latitude vs. Cloudiness

#### Latitude vs. Wind Speed

### Apply Linear Regression to Each Relationship